In [1]:
import os
import faiss
import json
import glob

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [16]:
os.environ["OpenAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model ="gpt-4o")

In [17]:
# folder_path = "food_recipe/"

# # 병합된 데이터를 저장할 리스트
# merged_data = []

# # 폴더 내 모든 JSON 파일 읽기
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".json"):  # JSON 파일만 처리
#         file_path = os.path.join(folder_path, file_name)
#         with open(file_path, 'r', encoding='utf-8') as file:
#             try:
#                 # JSON 데이터 불러오기
#                 data = json.load(file)
#                 merged_data.append(data)  # 병합 리스트에 추가
#                 print(f"Added data from {file_name}")
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding {file_name}: {e}")

# # 병합된 데이터를 하나의 JSON 파일로 저장
# output_path = os.path.join(folder_path, "final_recipes_data.json")
# with open(output_path, 'w', encoding='utf-8') as output_file:
#     json.dump(merged_data, output_file, indent=4, ensure_ascii=False)

# print(f"병합된 JSON 데이터가 {output_path}에 저장되었습니다.")


In [18]:
# with open("food_recipe/food_recipes_1-68.json",'r',encoding='utf-8') as file:
#     data = json.load(file)

In [19]:
# document_recipe = [
#     Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
#     for recipe in data
# ]

In [20]:
# document_recipe

In [21]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [22]:
# index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

# recipe_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={}
# )

In [23]:
# uuids = [str(uuid4()) for _ in range(len(document_recipe))]

# recipe_store.add_documents(documents=document_recipe, ids = uuids)

In [25]:
# recipe_store.save_local("./food_db")
recipes = FAISS.load_local("./food_db" ,embeddings, allow_dangerous_deserialization=True)

In [26]:
retriever = recipes.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [31]:
prompt = ChatPromptTemplate.from_template("""
너는 사용자가 물어보는 음식의 레시피를 자세하게 알려주는 요리사야.
친절하고 전문적인 말투로 응답해.
데이터에 포함되어 있는 레시피만을 제공하도록 해.
해당 음식이 데이터에 포함되어 있지 않다면 양해를 구하고 관련 데이터를 제공할 수 없음을 명시해줘.
대신 해당 음식과 비슷한 재료가 들어가있는 음식 리스트를 최대 5개 소개해줘.
기본적으로 해당 레시피의 원래 인분 수로 제공하고, 사용자가 원하는 인분 수에 따라 재료의 양을 계산해서 수정해줘.

'''
user : <음식명3> 을 만드는 법을 알려줘
ai : <음식명3> 을 만드는 방법은 아래와 같습니다:
- 요리의 이름
- 재료
    - 재료 1 (재료양)
    - 재료 2 (재료양)
    - 재료 3 (재료양)
- 조리 시간
- 인분 수
- 요리 방법
    - 첫번째, ~~
    - 두번째, ~~
    - 세번째, ~~
'''

'''
user: 김치우동을 만드는법을 알려줘
ai: 죄송합니다. 김치우동에 관련한 정보는 제가 가져올 수 없습니다. 
대신 김치우동과 비슷한 재료가 들어가는 음식들을 소개해드리겠습니다:
- 요리명1
  - 필요 재료
  - 조리 시간: 15분
- 요리명2
  - 필요 재료
  - 조리 시간: 10분
~~

'''
'''
user : 1인분의 레시피를 알려줘
ai : <음식명>의 레시피는 ~ 인분 기준입니다. 
<음식명> 을 1인분으로 만드는 레시피는 다음과 같습니다:
- 재료 : 1인분으로 수정된 재료 나열
- 요리 방법 : 요리 방법 나열
'''

다음과 같은 데이터를 학습해:\n{data}"
그리고 질문에 답해:\n{question}

""")

In [32]:
contextual_prompt = prompt

In [33]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        return {"data": inputs["data"], "question": inputs["question"]}
    
rag_chain_divide = {
    "data" : retriever,
    "question" : DebugPassThrough()

# contextual_prompt에 기능을 분리시키는 프롬프트 입력 (~라면 a를 반환, ~라면 b를 반환)
} | DebugPassThrough() | ContextToText()| contextual_prompt | model 


#위의 반환된 값에 따라서 레시피를 뽑을지, 음식을 추천 할 지에 대해서 나눔.
#그에 따라서  contextual_prompt의 변경이 필요함.
rag_chain_progress = {
    "data" : retriever,
    "question" : DebugPassThrough()
} | DebugPassThrough() | ContextToText()| contextual_prompt | model


In [34]:
while True:
    print("-----------------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_divide.invoke(query)
    
    
    
    print("Question : ", query)
    print(response.content)

-----------------------------
Question :  김치 요리를 알려줘
김치 요리를 만드는 방법은 아래와 같습니다:

- 요리의 이름: 김치만두
- 재료:
  - 배추김치: 2/3컵 (400g)
  - 찹쌀가루: 1컵 (100g)
  - 밀가루: 1컵 (100g)
  - 굴: 250g
  - 메밀가루: 1컵 (100g)
  - 들기름: 1큰술 (15ml)
  - 물: 1/2컵 (100ml)
- 조리 시간: 60분
- 인분 수: 4인분 기준
- 요리 방법:
  - 첫번째, 찹쌀가루와 메밀가루를 섞은 후 김이 오른 찜기에 젖은 면보를 깔고 찹쌀가루와 메밀가루를 넣고 찐다.
  - 두번째, 볼에 찐 가루들과 밀가루를 넣고 물을 넣어가며 치대어 반죽하고, 반죽을 떼어내 둥글게 빚어 밀대로 밀어 지름 8cm 크기로 만든다.
  - 세번째, 굴을 소금물에 넣고 살살 흔들어 씻은 후 체에 밭쳐 헹구고 물기를 뺀다.
  - 네번째, 배추김치는 속을 털어내고 다진 후 볼에 배추김치, 굴, 들기름을 넣고 골고루 섞는다.
  - 다섯번째, 만두피에 소를 올리고 가장자리에 물을 묻힌 후 반으로 접어 잘 눌러 붙인다.
  - 여섯번째, 만두를 김이 오른 찜기에 올려 찐다.

이 외에도 다른 김치 요리들이 있습니다. 원하시면 다른 요리의 레시피도 제공해드리겠습니다!
-----------------------------
Question :  다른 김치 요리를 알려줘
김치 요리를 찾고 계시군요! 아래에 다양한 김치를 활용한 요리를 소개해드리겠습니다:

1. 김치만두
   - 필요 재료: 배추김치, 찹쌀가루, 밀가루, 굴, 메밀가루, 들기름, 물 등
   - 조리 시간: 60분

2. 김치볶음밥
   - 필요 재료: 김치, 밥, 쇠고기, 양파, 당근, 올리브유, 달걀 등
   - 조리 시간: 30분

3. 묵은지김치찜
   - 필요 재료: 묵은지, 돼지고기, 김치국물, 멸치육수, 청주, 대파, 마늘 등
   - 조리 시간: 60분

4. 김치참치볶음
   - 필요 재

## 문제점
 
 * 이전 대화 내용을 기억 못 함

* 나만의 문제일수도 있는데 대화를 좀 진행하고 나서 질문을 입력했는데도 출력 안 나오고 다시 질문 입력하라 했음
  * 다시 입력하면 한번에 각각의 질문과 출력이 함께 나옴

-----
## 해결 된 것
-  없는 음식 데이터를 들고 오지 않음
-  